# SQL query from table names - Continued

In [1]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
from openai import OpenAI
import os

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [3]:
print("OpenAI Key:", "Loaded ✅" if OPENAI_API_KEY else "Missing ❌")


OpenAI Key: Loaded ✅


## The old Prompt

In [4]:
#The old prompt
old_context = [ {'role':'system', 'content':"""
you are a bot to assist in create SQL commands, all your answers should start with \
this is your SQL, and after that an SQL that can do what the user request. \
Your Database is composed by a SQL database with some tables. \
Try to maintain the SQL order simple.
Put the SQL command in white letters with a black background, and just after \
a simple and concise text explaining how it works.
If the user ask for something that can not be solved with an SQL Order \
just answer something nice and simple, maximum 10 words, asking him for something that \
can be solved with SQL.
"""} ]

old_context.append( {'role':'system', 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "nombre": "ID_usr",
      "tipo": "int"
    },
    {
      "nombre": "name",
      "tipo": "varchar"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "nombre": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

old_context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational_level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "varchar"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "varchar"
    }
  ]
}
"""
})

## New Prompt.
We are going to improve it following the instructions of a Paper from the Ohaio University: [How to Prompt LLMs for Text-to-SQL: A Study in Zero-shot, Single-domain, and Cross-domain Settings](https://arxiv.org/abs/2305.11853). I recommend you read that paper.

For each table, we will define the structure using the same syntax as in a SQL create table command, and add the sample rows of the content.

Finally, at the end of the prompt, we'll include some example queries with the SQL that the model should generate. This technique is called Few-Shot Samples, in which we provide the prompt with some examples to assist it in generating the correct SQL.


In [5]:
#context = [ {'role':'system', 'content':"""
# CREATE SEVERAL (3+) TABLES HERE
#"""} ]

context = []

# Example Table 1: Customers
context.append( {'role':'system', 'content':"""
 CREATE TABLE Customers (
    CustomerID INT PRIMARY KEY,
    CustomerName VARCHAR(255),
    ContactName VARCHAR(255),
    Country VARCHAR(255)
 );

 INSERT INTO Customers (CustomerID, CustomerName, ContactName, Country) VALUES
 (1, 'Alfreds Futterkiste', 'Maria Anders', 'Germany'),
 (2, 'Ana Trujillo Emparedados y helados', 'Ana Trujillo', 'Mexico'),
 (3, 'Antonio Moreno Taquería', 'Antonio Moreno', 'Mexico'),
 (4, 'Around the Horn', 'Thomas Hardy', 'UK'),
 (5, 'Berglunds snabbköp', 'Christina Berglund', 'Sweden');
"""} )

# Example Table 2: Products
context.append( {'role':'system', 'content':"""
 CREATE TABLE Products (
    ProductID INT PRIMARY KEY,
    ProductName VARCHAR(255),
    SupplierID INT,
    CategoryID INT,
    Price DECIMAL(10, 2)
 );

 INSERT INTO Products (ProductID, ProductName, SupplierID, CategoryID, Price) VALUES
 (1, 'Chais', 1, 1, 18.00),
 (2, 'Chang', 1, 1, 19.00),
 (3, 'Aniseed Syrup', 1, 2, 10.00),
 (4, 'Chef Anton''s Cajun Seasoning', 2, 2, 22.00),
 (5, 'Chef Anton''s Gumbo Mix', 2, 2, 21.35);
"""} )

# Example Table 3: Orders
context.append( {'role':'system', 'content':"""
 CREATE TABLE Orders (
    OrderID INT PRIMARY KEY,
    CustomerID INT,
    EmployeeID INT,
    OrderDate DATE,
    ShipperID INT
 );

 INSERT INTO Orders (OrderID, CustomerID, EmployeeID, OrderDate, ShipperID) VALUES
 (10248, 1, 5, '1996-07-04', 3),
 (10249, 3, 6, '1996-07-05', 1),
 (10250, 4, 4, '1996-07-08', 2),
 (10251, 5, 3, '1996-07-08', 2),
 (10252, 1, 3, '1996-07-09', 3);
"""} )

# Example Table 4: Shippers
context.append( {'role':'system', 'content':"""
 CREATE TABLE Shippers (
    ShipperID INT PRIMARY KEY,
    CompanyName VARCHAR(255),
    Phone VARCHAR(255)
 );

 INSERT INTO Shippers (ShipperID, CompanyName, Phone) VALUES
 (1, 'Speedy Express', '(503) 555-9831'),
 (2, 'United Package', '(503) 555-3199'),
 (3, 'Federal Shipping', '(503) 555-9931');
"""} )



In [6]:
#FEW SHOT SAMPLES
context.append( {'role':'system', 'content':"""
 -- Maintain the SQL order simple and efficient as you can, using valid SQL Lite, answer the following questions for the table provided above.
 -- Example 1:
 -- User: List the names of all customers from Germany.
 -- SQL: SELECT CustomerName FROM Customers WHERE Country = 'Germany';
 -- Example 2:
 -- User: What is the price of the product 'Chang'?
 -- SQL: SELECT Price FROM Products WHERE ProductName = 'Chang';
 -- Example 3:
 -- User: Find the CustomerID and OrderDate for all orders placed by CustomerID 1.
 -- SQL: SELECT CustomerID, OrderDate FROM Orders WHERE CustomerID = 1;
 -- Example 4:
 -- User: What is the name of the shipper with ShipperID 2?
 -- SQL: SELECT CompanyName FROM Shippers WHERE ShipperID = 2;
"""
})

In [7]:
#Functio to call the model.
def return_CCRMSQL(user_message, context):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)

    newcontext = context.copy()
    newcontext.append({'role':'user', 'content':"question: " + user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=newcontext,
            temperature=0,
        )

    return (response.choices[0].message.content)

## NL2SQL Samples
We're going to review some examples generated with the old prompt and others with the new prompt.

In [8]:
#new
context_user = context.copy()
print(return_CCRMSQL("List the names of all products with a price greater than 20.", context_user))

```sql
SELECT ProductName FROM Products WHERE Price > 20;
```


In [9]:
#old
old_context_user = old_context.copy()
print(return_CCRMSQL("Find the customer name for OrderID 10250", old_context_user))

This is your SQL:
```sql
SELECT name
FROM employees
WHERE ID_usr = 10250;
```
This SQL query selects the name of the customer from the employees table where the ID_usr matches the given OrderID 10250.


In [10]:
#new
print(return_CCRMSQL("YOUR QUERY HERE", context_user))

User: List the names of all customers from Mexico.


In [11]:
#old
print(return_CCRMSQL("YOUR QUERY HERE", old_context_user))

This is your SQL:
```sql

Please provide a specific query to assist you.


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try at least 3 versions
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong.
     - What did you learn?

In [ ]:
# Prompt Variation 1: More explicit instructions
context_v1 = []

# Example Table 1: Customers
context_v1.append( {'role':'system', 'content':"""
 CREATE TABLE Customers (
    CustomerID INT PRIMARY KEY,
    CustomerName VARCHAR(255),
    ContactName VARCHAR(255),
    Country VARCHAR(255)
 );

 INSERT INTO Customers (CustomerID, CustomerName, ContactName, Country) VALUES
 (1, 'Alfreds Futterkiste', 'Maria Anders', 'Germany'),
 (2, 'Ana Trujillo Emparedados y helados', 'Ana Trujillo', 'Mexico'),
 (3, 'Antonio Moreno Taquería', 'Antonio Moreno', 'Mexico'),
 (4, 'Around the Horn', 'Thomas Hardy', 'UK'),
 (5, 'Berglunds snabbköp', 'Christina Berglund', 'Sweden');
"""} )

# Example Table 2: Products
context_v1.append( {'role':'system', 'content':"""
 CREATE TABLE Products (
    ProductID INT PRIMARY KEY,
    ProductName VARCHAR(255),
    SupplierID INT,
    CategoryID INT,
    Price DECIMAL(10, 2)
 );

 INSERT INTO Products (ProductID, ProductName, SupplierID, CategoryID, Price) VALUES
 (1, 'Chais', 1, 1, 18.00),
 (2, 'Chang', 1, 1, 19.00),
 (3, 'Aniseed Syrup', 1, 2, 10.00),
 (4, 'Chef Anton''s Cajun Seasoning', 2, 2, 22.00),
 (5, 'Chef Anton''s Gumbo Mix', 2, 2, 21.35);
"""} )

# Example Table 3: Orders
context_v1.append( {'role':'system', 'content':"""
 CREATE TABLE Orders (
    OrderID INT PRIMARY KEY,
    CustomerID INT,
    EmployeeID INT,
    OrderDate DATE,
    ShipperID INT
 );

 INSERT INTO Orders (OrderID, CustomerID, EmployeeID, OrderDate, ShipperID) VALUES
 (10248, 1, 5, '1996-07-04', 3),
 (10249, 3, 6, '1996-07-05', 1),
 (10250, 4, 4, '1996-07-08', 2),
 (10251, 5, 3, '1996-07-08', 2),
 (10252, 1, 3, '1996-07-09', 3);
"""} )

# Example Table 4: Shippers
context_v1.append( {'role':'system', 'content':"""
 CREATE TABLE Shippers (
    ShipperID INT PRIMARY KEY,
    CompanyName VARCHAR(255),
    Phone VARCHAR(255)
 );

 INSERT INTO Shippers (ShipperID, CompanyName, Phone) VALUES
 (1, 'Speedy Express', '(503) 555-9831'),
 (2, 'United Package', '(503) 555-3199'),
 (3, 'Federal Shipping', '(503) 555-9931');
"""} )

#FEW SHOT SAMPLES
context_v1.append( {'role':'system', 'content':"""
 -- You are a helpful assistant that translates natural language questions into SQL queries based on the provided table schemas and data.
 -- Generate simple and efficient SQL Lite queries.
 -- Answer the following questions for the table provided above.
 -- Example 1:
 -- User: List the names of all customers from Germany.
 -- SQL: SELECT CustomerName FROM Customers WHERE Country = 'Germany';
 -- Example 2:
 -- User: What is the price of the product 'Chang'?
 -- SQL: SELECT Price FROM Products WHERE ProductName = 'Chang';
 -- Example 3:
 -- User: Find the CustomerID and OrderDate for all orders placed by CustomerID 1.
 -- SQL: SELECT CustomerID, OrderDate FROM Orders WHERE CustomerID = 1;
 -- Example 4:
 -- User: What is the name of the shipper with ShipperID 2?
 -- SQL: SELECT CompanyName FROM Shippers WHERE ShipperID = 2;
"""
})


# Prompt Variation 2: Focus on simplicity
context_v2 = []

# Example Table 1: Customers
context_v2.append( {'role':'system', 'content':"""
 CREATE TABLE Customers (
    CustomerID INT PRIMARY KEY,
    CustomerName VARCHAR(255),
    ContactName VARCHAR(255),
    Country VARCHAR(255)
 );

 INSERT INTO Customers (CustomerID, CustomerName, ContactName, Country) VALUES
 (1, 'Alfreds Futterkiste', 'Maria Anders', 'Germany'),
 (2, 'Ana Trujillo Emparedados y helados', 'Ana Trujillo', 'Mexico'),
 (3, 'Antonio Moreno Taquería', 'Antonio Moreno', 'Mexico'),
 (4, 'Around the Horn', 'Thomas Hardy', 'UK'),
 (5, 'Berglunds snabbköp', 'Christina Berglund', 'Sweden');
"""} )

# Example Table 2: Products
context_v2.append( {'role':'system', 'content':"""
 CREATE TABLE Products (
    ProductID INT PRIMARY KEY,
    ProductName VARCHAR(255),
    SupplierID INT,
    CategoryID INT,
    Price DECIMAL(10, 2)
 );

 INSERT INTO Products (ProductID, ProductName, SupplierID, CategoryID, Price) VALUES
 (1, 'Chais', 1, 1, 18.00),
 (2, 'Chang', 1, 1, 19.00),
 (3, 'Aniseed Syrup', 1, 2, 10.00),
 (4, 'Chef Anton''s Cajun Seasoning', 2, 2, 22.00),
 (5, 'Chef Anton''s Gumbo Mix', 2, 2, 21.35);
"""} )

# Example Table 3: Orders
context_v2.append( {'role':'system', 'content':"""
 CREATE TABLE Orders (
    OrderID INT PRIMARY KEY,
    CustomerID INT,
    EmployeeID INT,
    OrderDate DATE,
    ShipperID INT
 );

 INSERT INTO Orders (OrderID, CustomerID, EmployeeID, OrderDate, ShipperID) VALUES
 (10248, 1, 5, '1996-07-04', 3),
 (10249, 3, 6, '1996-07-05', 1),
 (10250, 4, 4, '1996-07-08', 2),
 (10251, 5, 3, '1996-07-08', 2),
 (10252, 1, 3, '1996-07-09', 3);
"""} )

# Example Table 4: Shippers
context_v2.append( {'role':'system', 'content':"""
 CREATE TABLE Shippers (
    ShipperID INT PRIMARY KEY,
    CompanyName VARCHAR(255),
    Phone VARCHAR(255)
 );

 INSERT INTO Shippers (ShipperID, CompanyName, Phone) VALUES
 (1, 'Speedy Express', '(503) 555-9831'),
 (2, 'United Package', '(503) 555-3199'),
 (3, 'Federal Shipping', '(503) 555-9931');
"""} )


#FEW SHOT SAMPLES
context_v2.append( {'role':'system', 'content':"""
 -- Generate concise SQL Lite queries for the given question based on the provided tables.
 -- Example 1:
 -- User: Customers from Germany.
 -- SQL: SELECT CustomerName FROM Customers WHERE Country = 'Germany';
 -- Example 2:
 -- User: Price of 'Chang'.
 -- SQL: SELECT Price FROM Products WHERE ProductName = 'Chang';
 -- Example 3:
 -- User: Order details for CustomerID 1.
 -- SQL: SELECT CustomerID, OrderDate FROM Orders WHERE CustomerID = 1;
 -- Example 4:
 -- User: Shipper name for ID 2.
 -- SQL: SELECT CompanyName FROM Shippers WHERE ShipperID = 2;
"""
})


# Prompt Variation 3: Adding a persona
context_v3 = []

# Example Table 1: Customers
context_v3.append( {'role':'system', 'content':"""
 CREATE TABLE Customers (
    CustomerID INT PRIMARY KEY,
    CustomerName VARCHAR(255),
    ContactName VARCHAR(255),
    Country VARCHAR(255)
 );

 INSERT INTO Customers (CustomerID, CustomerName, ContactName, Country) VALUES
 (1, 'Alfreds Futterkiste', 'Maria Anders', 'Germany'),
 (2, 'Ana Trujillo Emparedados y helados', 'Ana Trujillo', 'Mexico'),
 (3, 'Antonio Moreno Taquería', 'Antonio Moreno', 'Mexico'),
 (4, 'Around the Horn', 'Thomas Hardy', 'UK'),
 (5, 'Berglunds snabbköp', 'Christina Berglund', 'Sweden');
"""} )

# Example Table 2: Products
context_v3.append( {'role':'system', 'content':"""
 CREATE TABLE Products (
    ProductID INT PRIMARY KEY,
    ProductName VARCHAR(255),
    SupplierID INT,
    CategoryID INT,
    Price DECIMAL(10, 2)
 );

 INSERT INTO Products (ProductID, ProductName, SupplierID, CategoryID, Price) VALUES
 (1, 'Chais', 1, 1, 18.00),
 (2, 'Chang', 1, 1, 19.00),
 (3, 'Aniseed Syrup', 1, 2, 10.00),
 (4, 'Chef Anton''s Cajun Seasoning', 2, 2, 22.00),
 (5, 'Chef Anton''s Gumbo Mix', 2, 2, 21.35);
"""} )

# Example Table 3: Orders
context_v3.append( {'role':'system', 'content':"""
 CREATE TABLE Orders (
    OrderID INT PRIMARY KEY,
    CustomerID INT,
    EmployeeID INT,
    OrderDate DATE,
    ShipperID INT
 );

 INSERT INTO Orders (OrderID, CustomerID, EmployeeID, OrderDate, ShipperID) VALUES
 (10248, 1, 5, '1996-07-04', 3),
 (10249, 3, 6, '1996-07-05', 1),
 (10250, 4, 4, '1996-07-08', 2),
 (10251, 5, 3, '1996-07-08', 2),
 (10252, 1, 3, '1996-07-09', 3);
"""} )

# Example Table 4: Shippers
context_v3.append( {'role':'system', 'content':"""
 CREATE TABLE Shippers (
    ShipperID INT PRIMARY KEY,
    CompanyName VARCHAR(255),
    Phone VARCHAR(255)
 );

 INSERT INTO Shippers (ShipperID, CompanyName, Phone) VALUES
 (1, 'Speedy Express', '(503) 555-9831'),
 (2, 'United Package', '(503) 555-3199'),
 (3, 'Federal Shipping', '(503) 555-9931');
"""} )


#FEW SHOT SAMPLES
context_v3.append( {'role':'system', 'content':"""
 -- You are a SQL expert. Generate the most efficient SQL Lite query for the given question based on the provided tables and data.
 -- Example 1:
 -- User: List the names of all customers from Germany.
 -- SQL: SELECT CustomerName FROM Customers WHERE Country = 'Germany';
 -- Example 2:
 -- User: What is the price of the product 'Chang'?
 -- SQL: SELECT Price FROM Products WHERE ProductName = 'Chang';
 -- Example 3:
 -- User: Find the CustomerID and OrderDate for all orders placed by CustomerID 1.
 -- SQL: SELECT CustomerID, OrderDate FROM Orders WHERE CustomerID = 1;
 -- Example 4:
 -- User: What is the name of the shipper with ShipperID 2?
 -- SQL: SELECT CompanyName FROM Shippers WHERE ShipperID = 2;
"""
})